импорты и константы

In [23]:
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
import re
import concurrent.futures
import shutil
import pandas as pd
import time

In [51]:
# 153.8336486816406250 884.2712384333286764    измеряемый диапазон. 0-2136 диапазон данных
start_nm = 500  # нм
end_nm = 650  # нм

x = pd.read_excel("x points nm.xlsx")
x = np.array(x["nm"].values[1 : len(x) - 1])

def nm_to_x(nm):
    return xint.index(nm) + 1
def x_to_nm(index):
    return x[index]


# номера точек в массиве по координате х
for i in range(len(x)):
    if x[i] > start_nm:
        start_x = i
        break
for i in range(len(x) - 1, 0, -1):
    if x[i] < end_nm:
        end_x = i
        break
x = x[start_x:end_x]
xint = list(np.array(x, dtype=int))


получение максимумов из спектров

In [52]:
main_folder = r"D:\Data\new test\spectra3"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
print(main_folder)
file_list = np.array(natsorted(os.listdir(main_folder)))
print(len(file_list), " файлов")  # изанчальные графики

data_max = np.zeros(len(file_list))
data_tail = np.zeros(len(file_list))

h=len(file_list)//12
q=len(file_list)%12
m=[]
for i in range(12):
    if i==11:
        m.append([h*i,h*(i+1)+q])
    else:
        m.append([h*i,h*(i+1)])
print(m)

def get_max_rmr(op):
    for i in range(op[0],op[1]):
        with open(main_folder + file_list[i], "r", encoding="utf8") as spec:
            spec = re.split(",", spec.read())
            d= np.array(spec[start_x:end_x], dtype=float)
            data_max[i]=np.max(d)
            data_tail[i]=data_max[i]-d[440]

    


with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(get_max_rmr, m)

print("получено " + str(len(data_max)) + " максимумов спектров")

D:/Data/new test/spectra3/
137926  файлов
[[0, 11493], [11493, 22986], [22986, 34479], [34479, 45972], [45972, 57465], [57465, 68958], [68958, 80451], [80451, 91944], [91944, 103437], [103437, 114930], [114930, 126423], [126423, 137926]]
получено 137926 максимумов спектров


максимумы на таймлайнах

In [53]:
_step = 400
_sp = 0
_ep = _step

# создание папки для сохранения картинок
dist_folder = r"D:\Data\new test"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"
folder_name = "t3"
if not os.path.exists(os.path.join(dist_folder, folder_name)):
    os.makedirs(os.path.join(dist_folder, folder_name))

# расчет гистограммы
num_bins = 10  # количество интервалов
range_values = (np.max(data_max) - np.min(data_max)) / num_bins
# границы интервалов
bin_edges = np.arange(np.min(data_max), np.max(data_max) + range_values, range_values)
# середины интервалов
bins = bin_edges[:-1] + range_values / 2

hist, _ = np.histogram(data_max, bins=bin_edges)

zero = bin_edges[np.argmax(hist) + 1]
fig, ax = plt.subplots(figsize=(10, 6), dpi=120)
fig.suptitle("гистограмма")
ax.hist(data_max, bin_edges, alpha=0.6, color="blue", edgecolor="k", linewidth=1)
ax.plot(bins, hist, color="red", lw=2)
plt.xticks(rotation=90)
ax.set_xticks(bin_edges)
plt.savefig(dist_folder + "/" + folder_name + "/" + "гистограмма.png")

m = np.min(data_max) * 0.8
m2 = np.max(data_max) * 0.5

for i in range(round(len(data_max) / (3 * _step))):
    fig, axs = plt.subplots(3, 1, figsize=(10, 6), dpi=120)
    plt.ioff()
    _s = _sp
    for j in range(3):
        if _ep > len(data_max):
            _ep = len(data_max)
        axs[j].plot(np.arange(_sp, _ep, 1), data_max[_sp:_ep], lw=1, color="blue")
        axs[j].scatter(np.arange(_sp, _ep, 1), data_max[_sp:_ep], s=4, color="black")
        axs[j].scatter(np.arange(_sp, _ep, 1), data_tail[_sp:_ep], s=4, color="red")
        axs[j].plot((_sp, _ep), (zero, zero), color="orange")
        axs[j].set_ylim(0, 0.5)
        _sp += _step
        _ep += _step

    _e = _ep - _step
    fig.suptitle("спектры " + str(_s) + "-" + str(_e))
    plt.savefig(
        dist_folder
        + "/"
        + folder_name
        + "/"
        + "ряд максимумов "
        + str(_s)
        + "-"
        + str(_e)
        + ".png"
    )

In [7]:
point_index = []
_point = False
k = []
zero = 0.05
s = 500
for i in range(0, len(data_max), s):
    ar = np.mean(data_max[i : i + s])
    if ar > zero:
        k.append(i)
        _point = True
    if ar < zero and _point == True:
        point_index.append((k[0], i))
        k = []
        _point = False
    if _point == True and i >= len(data_max) - s:
        point_index.append((k[0], len(data_max) - 1))
print(len(point_index))
print(point_index)

7
[(10500, 12000), (14500, 16500), (19500, 21500), (24000, 24500), (43000, 44000), (46000, 47500), (49500, 51000)]


In [55]:
for i in range(len(point_index)):
    print(i+1, ' ',point_index[i], "   ", point_index[i][1] - point_index[i][0])

1   [3900, 8700]     4800
2   [11400, 15850]     4450
3   [18800, 24100]     5300
4   [26900, 31500]     4600
5   [34350, 39150]     4800
6   [46550, 51350]     4800
7   [54000, 58800]     4800
8   [61400, 66550]     5150
9   [69450, 74000]     4550
10   [76850, 81400]     4550
11   [88900, 93800]     4900
12   [96450, 101200]     4750
13   [103800, 108800]     5000
14   [111800, 116100]     4300
15   [118500, 122500]     4000
16   [128900, 132950]     4050


In [54]:
point_index = [
[3900,8700],
[11400,15850],
[18800,24100],
[26900,31500],
[34350,39150],

[46550,51350],
[54000,58800],
[61400,66550],
[69450,74000],
[76850,81400],

[88900,93800],
[96450,101200],
[103800,108800],
[111800,116100],
[118500,122500],
[128900,132950]
]

In [56]:
dist_folder = r"D:\Data\new test"
dist_folder = dist_folder.replace(chr(92), "/")
dist_folder += "/"

folder_name = "p3"
d_name = "3_"


if os.path.exists(os.path.join(dist_folder, folder_name)):

    shutil.rmtree(dist_folder + "/" + folder_name)


os.makedirs(os.path.join(dist_folder, folder_name))


for i in range(len(point_index)):

    os.makedirs(os.path.join(dist_folder + "/" + folder_name, d_name + str(i + 1)))

In [57]:
main_folder = r"D:\Data\new test\spectra3"
main_folder = main_folder.replace(chr(92), "/")
main_folder += "/"
file_list = np.array(natsorted(os.listdir(main_folder)))
print(dist_folder)


for i in range(len(point_index)):

    dist = dist_folder + "/" + folder_name + "/" + d_name + str(i + 1)

    for j in range(point_index[i][0], point_index[i][1]):

        shutil.copy2(main_folder + str(file_list[j]), dist)

    print(d_name, i + 1, " ", point_index[i][1] - point_index[i][0])

D:/Data/new test/
3_ 1   4800
3_ 2   4450
3_ 3   5300
3_ 4   4600
3_ 5   4800
3_ 6   4800
3_ 7   4800
3_ 8   5150
3_ 9   4550
3_ 10   4550
3_ 11   4900
3_ 12   4750
3_ 13   5000
3_ 14   4300
3_ 15   4000
3_ 16   4050
